#Data Preparation

In [1]:
from glob import glob
import os
import cv2 as cv
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
import tifffile
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Augmentation

In [ ]:
folder_path = '/content/drive/MyDrive/Data/Slash/'


datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)


for filename in os.listdir(folder_path):

    image_path = os.path.join(folder_path, filename)
    image = cv.imread(image_path)
    image = cv.cvtColor(image, cv.COLOR_BGR2RGB)

    if image is not None:
        image = image.reshape((1,) + image.shape)

        augment_iter = datagen.flow(image, batch_size=1, save_to_dir=folder_path, save_prefix='aug_', save_format='jpg')

        num_augmented_images = 10
        for _ in range(num_augmented_images):
            next(augment_iter)
    else:
        print("Error loading image:", image_path)


## Data Loading and Dataset Creation

In [2]:
def load_dataset(folder_path):
    images = []
    labels_str = []
    labels_names=[]

    for class_name in os.listdir(folder_path):
        class_folder = os.path.join(folder_path, class_name)

        if os.path.isdir(class_folder):
            print("Processing class:", class_name)

            for image_name in os.listdir(class_folder):
                image_path = os.path.join(class_folder, image_name)

                image = cv.imread(image_path)
                if image is not None:
                    image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
                    image = cv.resize(image, (256, 256))
                    images.append(image)
                    labels_str.append(class_name)
                else:
                    print("Error loading image:", image_path)

    images = np.array(images)/255.0
    labels_str = np.array(labels_str)

    #print(images.shape)
    #print(labels_str)

    label_to_index = {label: index for index, label in enumerate(np.unique(labels_str))}
    print(label_to_index)
    labels_names=list(label_to_index.keys())
    labels = np.array([label_to_index[label] for label in labels_str])
    #print(labels)
    # One-hot encode numerical labels
    num_classes = len(label_to_index)
    labels_one_hot = tf.one_hot(labels, num_classes)

    dataset = tf.data.Dataset.from_tensor_slices((images, labels_one_hot))

    return dataset, labels_names

In [3]:
folder_path='/content/drive/MyDrive/Data/Slash/web_scrapping'

In [ ]:
dataset, labels_names = load_dataset(folder_path)

In [ ]:
labels_names

In [6]:
dataset_length = tf.data.experimental.cardinality(dataset).numpy()

In [ ]:
dataset_length

In [8]:
dataset = dataset.shuffle(buffer_size=dataset_length)
dataset = dataset.shuffle(buffer_size=dataset_length)

In [9]:
def plot_images(dataset, num_images=5):
    plt.figure(figsize=(15, 5))
    for i, (image, label) in enumerate(dataset.take(num_images)):
        plt.subplot(1, num_images, i+1)
        plt.imshow(image.numpy())
        plt.title(f'Label: {labels_names[np.argmax(label.numpy())]}')
        plt.axis('off')
    plt.show()

In [ ]:
plot_images(dataset)

In [11]:
train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size

# Shuffle the dataset
dataset = dataset.shuffle(buffer_size=len(dataset))

train_dataset = dataset.take(train_size)
val_test_dataset = dataset.skip(train_size)
val_dataset = val_test_dataset.take(val_size)
test_dataset = val_test_dataset.skip(val_size)

batch_size = 4
epochs=20

num_classes=len(labels_names)
train_dataset = train_dataset.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
val_dataset = val_dataset.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
test_dataset = test_dataset.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
train_dataset

# Creating The Model

In [13]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

In [15]:
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=2,
    min_delta=0.001,
    mode='min',
    verbose=1
)

In [16]:
model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer=Adam(learning_rate=0.01), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(train_dataset, validation_data=val_dataset, epochs=epochs, batch_size=batch_size, callbacks=[early_stopping])

In [ ]:
test_loss, test_accuracy = model.evaluate(test_dataset)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

y_pred_prob = []
y_true = []

for x_batch, y_batch in test_dataset:
    batch_pred_prob = model.predict(x_batch)
    y_pred_prob.extend(batch_pred_prob)
    y_true.extend(tf.argmax(y_batch, axis=1).numpy())

y_pred = np.argmax(y_pred_prob, axis=1)

conf_matrix = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()

print(classification_report(y_true, y_pred))

In [20]:
# model_path = '/content/drive/MyDrive/Data/Slash/model'

# model.save(model_path+'model_file.h5')

In [30]:
img=cv.imread('/content/drive/MyDrive/Data/Slash/xxl-rblksn-granat-inc-original-imag88p8gvth4hmb.webp')
img=cv.resize(img,(256,256))
img=cv.cvtColor(img, cv.COLOR_BGR2RGB)

In [ ]:
plt.imshow(img)

In [32]:
img=tf.cast(img, tf.float32) / 255.0
img = tf.expand_dims(img, axis=0)

In [ ]:
predictions=model(img)
probabilities = tf.nn.softmax(predictions)
print(labels_names[np.argmax(probabilities.numpy())])

In [ ]:
labels_names

In [26]:
# # !zip -r /content/model.zip /content/model.pt

# from google.colab import files
# files.download("model.zip")